# Flood Event Data Processing

This notebook processes storm event CSV files to extract and filter flooding events.


In [2]:
import os
import pandas as pd
from pathlib import Path
from typing import List


## Loading initial data

### Loading data


In [3]:
def get_csv_files(data_dir: str) -> List[str]:
    csv_files = []
    data_path = Path(data_dir)
    
    if not data_path.exists():
        raise FileNotFoundError(f"Directory {data_dir} does not exist")
    
    for file in data_path.glob("*.csv"):
        csv_files.append(str(file))
    
    return sorted(csv_files)


In [4]:
def parse_csv_file(file_path: str) -> pd.DataFrame:
    try:
        df = pd.read_csv(file_path, low_memory=False)
        return df
    except Exception as e:
        print(f"Error parsing {file_path}: {e}")
        return pd.DataFrame()


### Filtering


In [4]:
def filter_flooding_events(df: pd.DataFrame, state: str = None) -> pd.DataFrame:
    if df.empty or 'EVENT_TYPE' not in df.columns:
        return pd.DataFrame()
    
    flood_keywords = ['flood', 'flash flood', 'coastal flood', 'lakeshore flood', 
                      'river flood', 'urban flood', 'small stream flood']
    
    mask = df['EVENT_TYPE'].str.lower().str.contains('|'.join(flood_keywords), 
                                                       case=False, 
                                                       na=False)    
    if state is not None:
        if 'STATE' not in df.columns:
            print(f"Warning: STATE column not found, cannot filter by state")
        else:
            state_mask = df['STATE'].str.upper() == state.upper()
            mask = mask & state_mask
    
    df = df[mask].copy()

    # Remove rows with invalid coordinates
    if 'BEGIN_LAT' in df.columns and 'BEGIN_LON' in df.columns:
        df['BEGIN_LAT'] = pd.to_numeric(df['BEGIN_LAT'], errors='coerce')
        df['BEGIN_LON'] = pd.to_numeric(df['BEGIN_LON'], errors='coerce')
        mask = (df['BEGIN_LAT'].notna()) & (df['BEGIN_LON'].notna()) & \
            (df['BEGIN_LAT'] != 0) & (df['BEGIN_LON'] != 0)
        
        filtered_df = df[mask].copy()
    else:
        filtered_df = df.copy()

    return filtered_df


In [5]:
def split_yearmonth(df: pd.DataFrame) -> pd.DataFrame:

    if df.empty or 'BEGIN_YEARMONTH' not in df.columns:
        return df
    
    df = df.copy()
    df['YEAR'] = df['BEGIN_YEARMONTH'].astype(str).str[:4].astype(int)
    df['MONTH'] = df['BEGIN_YEARMONTH'].astype(str).str[4:6].astype(int)
    
    return df


In [6]:
def select_columns(df: pd.DataFrame) -> pd.DataFrame:

    if df.empty:
        return pd.DataFrame()
    
    columns_to_keep = [
        'YEAR',
        'MONTH',
        'BEGIN_DAY',
        'BEGIN_TIME',
        'BEGIN_LAT',
        'BEGIN_LON',
        'STATE',
        'EVENT_TYPE',
        'FLOOD_CAUSE',
        'EVENT_NARRATIVE'
    ]
    

    available_columns = [col for col in columns_to_keep if col in df.columns]
    missing_columns = [col for col in columns_to_keep if col not in df.columns]
    
    if missing_columns:
        print(f"Warning: The following columns were not found: {missing_columns}")
    
    if not available_columns:
        print("Warning: None of the requested columns were found in the DataFrame")
        return pd.DataFrame()
    
    selected_df = df[available_columns].copy()
    return selected_df


In [ ]:
def modify_flooding_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    from datetime import datetime, timedelta
    def modify_date(row):
        try:
            original_date = datetime(int(row['YEAR']), int(row['MONTH']), int(row['BEGIN_DAY']))
            new_date = original_date - timedelta(days=10)
            return new_date
        except (ValueError, KeyError) as e:
            return None
    
    modified_dates = df.apply(modify_date, axis=1)
    df.loc[modified_dates.notna(), 'YEAR'] = modified_dates[modified_dates.notna()].apply(lambda x: x.year)
    df.loc[modified_dates.notna(), 'MONTH'] = modified_dates[modified_dates.notna()].apply(lambda x: x.month)
    df.loc[modified_dates.notna(), 'BEGIN_DAY'] = modified_dates[modified_dates.notna()].apply(lambda x: x.day)
    
    # Change event type to 'Normal'
    if 'EVENT_TYPE' in df.columns:
        df['EVENT_TYPE'] = 'Normal'
    
    # Change flood cause to empty string
    if 'FLOOD_CAUSE' in df.columns:
        df['FLOOD_CAUSE'] = ''
    
    # Change event narrative to empty string
    if 'EVENT_NARRATIVE' in df.columns:
        df['EVENT_NARRATIVE'] = ''
    return df
    

### Main processing


In [ ]:
def process_all_csv_files(raw_data_dir: str, state: str = None) -> pd.DataFrame:
    csv_files = get_csv_files(raw_data_dir)
    
    if not csv_files:
        print(f"No CSV files found in {raw_data_dir}")
        return pd.DataFrame()
    
    print(f"Found {len(csv_files)} CSV file(s) to process")
    if state:
        print(f"Filtering for state: {state.upper()}")
    
    all_flooding_events = []
    
    for csv_file in csv_files:
        print(f"Processing {os.path.basename(csv_file)}...")
        df = parse_csv_file(csv_file)
        
        if not df.empty:
            flooding_df = filter_flooding_events(df, state=state)
            if not flooding_df.empty:
                print(f"  Found {len(flooding_df)} flooding event(s)")
                all_flooding_events.append(flooding_df)
            else:
                print(f"  No flooding events found")
        else:
            print(f"  Failed to parse or file is empty")
    
    if all_flooding_events:
        combined_df = pd.concat(all_flooding_events, ignore_index=True)
        print(f"\nTotal flooding events found: {len(combined_df)}")
        
        combined_df = split_yearmonth(combined_df)
        selected_df = select_columns(combined_df)
        selected_df = modify_flooding_data(selected_df)
        return selected_df
    else:
        print("\nNo flooding events found in any files")
        return pd.DataFrame()


In [14]:
raw_data_dir = "raw_data"
state_filter = "TEXAS" 

print(f"Data directory: {raw_data_dir}")
print(f"State filter: {state_filter if state_filter else 'None (all states)'}")


Data directory: raw_data
State filter: TEXAS


In [15]:
non_flooding_events_df = process_all_csv_files(str(raw_data_dir), state=state_filter)


Found 25 CSV file(s) to process
Filtering for state: TEXAS
Processing StormEvents_details-ftp_v1.0_d2001_c20250520.csv...
  Found 44 flooding event(s)
Processing StormEvents_details-ftp_v1.0_d2002_c20250520.csv...
  No flooding events found
Processing StormEvents_details-ftp_v1.0_d2003_c20250520.csv...
  Found 45 flooding event(s)
Processing StormEvents_details-ftp_v1.0_d2004_c20250520.csv...
  Found 32 flooding event(s)
Processing StormEvents_details-ftp_v1.0_d2005_c20250520.csv...
  Found 374 flooding event(s)
Processing StormEvents_details-ftp_v1.0_d2006_c20250520.csv...
  Found 136 flooding event(s)
Processing StormEvents_details-ftp_v1.0_d2007_c20250520.csv...
  Found 1345 flooding event(s)
Processing StormEvents_details-ftp_v1.0_d2008_c20250520.csv...
  Found 243 flooding event(s)
Processing StormEvents_details-ftp_v1.0_d2009_c20250520.csv...
  Found 533 flooding event(s)
Processing StormEvents_details-ftp_v1.0_d2010_c20250520.csv...
  Found 416 flooding event(s)
Processing Storm

In [16]:
# Display summary information
if not non_flooding_events_df.empty:
    print(f"\nTotal rows: {len(non_flooding_events_df)}")
    print(f"\nColumns: {list(non_flooding_events_df.columns)}")
    print(f"\nFirst few rows:")
    display(non_flooding_events_df.head(10))
    
    print(f"\nDataFrame info:")
    non_flooding_events_df.info()
else:
    print("No flooding events found.")



Total rows: 9340

Columns: ['YEAR', 'MONTH', 'BEGIN_DAY', 'BEGIN_TIME', 'BEGIN_LAT', 'BEGIN_LON', 'STATE', 'EVENT_TYPE', 'FLOOD_CAUSE', 'EVENT_NARRATIVE']

First few rows:


,YEAR,MONTH,BEGIN_DAY,BEGIN_TIME,BEGIN_LAT,BEGIN_LON,STATE,EVENT_TYPE,FLOOD_CAUSE,EVENT_NARRATIVE
0,2001,3,6,140,32.53333,-96.66667,TEXAS,Normal,,
1,2001,4,29,15,33.73333,-102.78333,TEXAS,Normal,,
2,2001,5,1,125,31.46667,-97.71667,TEXAS,Normal,,
3,2001,5,1,128,33.08333,-97.13333,TEXAS,Normal,,
4,2001,5,1,1700,33.36667,-97.68333,TEXAS,Normal,,
5,2001,5,1,45,33.75000,-96.55000,TEXAS,Normal,,
6,2001,4,30,2330,32.05000,-97.20000,TEXAS,Normal,,
7,2001,5,1,0,32.03333,-97.13333,TEXAS,Normal,,
8,2001,6,2,2000,29.36667,-95.08333,TEXAS,Normal,,
9,2001,6,3,430,30.50000,-95.33333,TEXAS,Normal,,



DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9340 entries, 0 to 9339
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   YEAR             9340 non-null   int64  
 1   MONTH            9340 non-null   int64  
 2   BEGIN_DAY        9340 non-null   int64  
 3   BEGIN_TIME       9340 non-null   int64  
 4   BEGIN_LAT        9340 non-null   float64
 5   BEGIN_LON        9340 non-null   float64
 6   STATE            9340 non-null   object 
 7   EVENT_TYPE       9340 non-null   object 
 8   FLOOD_CAUSE      9340 non-null   object 
 9   EVENT_NARRATIVE  9340 non-null   object 
dtypes: float64(2), int64(4), object(4)
memory usage: 729.8+ KB


In [14]:
# get a random subset of 10000 rows from the non_flooding_events_df
non_flooding_events_df = non_flooding_events_df.sample(n=9500)
non_flooding_events_df.head()


,YEAR,MONTH,BEGIN_DAY,BEGIN_TIME,BEGIN_LAT,BEGIN_LON,STATE,EVENT_TYPE,FLOOD_CAUSE,EVENT_NARRATIVE
22180,2010,5,1,18,29.41000,-100.90000,TEXAS,Hail,NaN,NaN
32610,2015,4,11,2040,35.24000,-100.16000,TEXAS,Hail,NaN,Law enforcement received golf ball size hail 5...
150,2001,3,14,1410,30.10000,-96.08333,TEXAS,Hail,NaN,NaN
2098,2001,10,12,1945,33.21667,-97.15000,TEXAS,Thunderstorm Wind,NaN,Large tree limbs were blown off trees.
37062,2016,4,27,45,30.41000,-98.74000,TEXAS,Thunderstorm Wind,NaN,A thunderstorm produced wind gusts estimated a...


## Augment Data

In [17]:
!pip3 install openmeteo-requests
!pip3 install requests-cache retry-requests

### Augmenting flooding data using the [Open Meteo API](https://open-meteo.com/en/docs/climate-api?utm_source=chatgpt.com&daily=temperature_2m_mean,wind_speed_10m_mean,cloud_cover_mean,relative_humidity_2m_mean,dew_point_2m_mean,precipitation_sum,rain_sum,snowfall_sum,pressure_msl_mean,soil_moisture_0_to_10cm_mean&start_date=2020-01-01&end_date=2020-01-01&models=EC_Earth3P_HR#settings)

This is an open-sourced api that allows you to entire a lat, long, and date and it gives you data on the weather conditions of that area at the time. 

In [7]:
import openmeteo_requests
import requests_cache
from retry_requests import retry

def get_weather_data(lat: float, lon: float, date: str) -> dict:
    cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    url = "https://climate-api.open-meteo.com/v1/climate"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": date,
        "end_date": date,
        "models": "EC_Earth3P_HR",
        "daily": ["temperature_2m_mean", "wind_speed_10m_mean", "cloud_cover_mean", "relative_humidity_2m_mean", "dew_point_2m_mean", "precipitation_sum", "rain_sum", "snowfall_sum", "pressure_msl_mean", "soil_moisture_0_to_10cm_mean"],
        "utm_source": "chatgpt.com",
    }
    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]

    daily = response.Daily()
    daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
    daily_wind_speed_10m_mean = daily.Variables(1).ValuesAsNumpy()
    daily_cloud_cover_mean = daily.Variables(2).ValuesAsNumpy()
    daily_relative_humidity_2m_mean = daily.Variables(3).ValuesAsNumpy()
    daily_dew_point_2m_mean = daily.Variables(4).ValuesAsNumpy()
    daily_precipitation_sum = daily.Variables(5).ValuesAsNumpy()
    daily_rain_sum = daily.Variables(6).ValuesAsNumpy()
    daily_snowfall_sum = daily.Variables(7).ValuesAsNumpy()
    daily_pressure_msl_mean = daily.Variables(8).ValuesAsNumpy()
    daily_soil_moisture_0_to_10cm_mean = daily.Variables(9).ValuesAsNumpy()

    daily_data = {"date": date}

    daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
    daily_data["wind_speed_10m_mean"] = daily_wind_speed_10m_mean
    daily_data["cloud_cover_mean"] = daily_cloud_cover_mean
    daily_data["relative_humidity_2m_mean"] = daily_relative_humidity_2m_mean
    daily_data["dew_point_2m_mean"] = daily_dew_point_2m_mean
    daily_data["precipitation_sum"] = daily_precipitation_sum
    daily_data["rain_sum"] = daily_rain_sum
    daily_data["snowfall_sum"] = daily_snowfall_sum
    daily_data["pressure_msl_mean"] = daily_pressure_msl_mean
    daily_data["soil_moisture_0_to_10cm_mean"] = daily_soil_moisture_0_to_10cm_mean
    daily_data["elevation"] = response.Elevation()

    daily_dataframe = pd.DataFrame(data = daily_data)
    return daily_dataframe

In [17]:
# For every flooding event, augment the row with the weather data
# Add the weather data to the flooding data
# Save the augmented data to a new csv file

import time
from datetime import datetime

def augment_flooding_data_with_weather(flooding_df: pd.DataFrame) -> pd.DataFrame:
    augmented_rows = []
    total_rows = len(flooding_df)
    
    print(f"Augmenting {total_rows} flooding events with weather data...")
    print("This may take a while due to API rate limits...\n")
    
    count = 0
    for _, row in flooding_df.iterrows():
        try:
            year = int(row['YEAR'])
            month = int(row['MONTH'])
            day = int(row['BEGIN_DAY'])
            date_str = f"{year}-{month:02d}-{day:02d}"
            
            weather_df = get_weather_data(
                lat=float(row['BEGIN_LAT']),
                lon=float(row['BEGIN_LON']),
                date=date_str
            )
            
            if not weather_df.empty:
                weather_row = weather_df.iloc[0]
                augmented_row = row.to_dict()
                
                for col in weather_df.columns:
                    if col != 'date':
                        augmented_row[col] = weather_row[col]
                augmented_rows.append(augmented_row)
            else:
                augmented_rows.append(row.to_dict())
            
            if (count + 1) % 50 == 0:
                print(f"Processed {count + 1}/{total_rows} events...")
            time.sleep(0.1)
            
        except Exception as e:
            print(f"Error processing row {count}: {e}")
            augmented_rows.append(row.to_dict())
        count += 1
    augmented_df = pd.DataFrame(augmented_rows)
    
    print(f"\nAugmentation complete! Processed {len(augmented_df)} events.")
    return augmented_df

# Augment the flooding events with weather data
augmented_non_flooding_df = augment_flooding_data_with_weather(non_flooding_events_df)

# Display summary
print(f"\nAugmented DataFrame shape: {augmented_non_flooding_df.shape}")
print(f"\nNew columns added: {set(augmented_non_flooding_df.columns) - set(augmented_non_flooding_df.columns)}")
print(f"\nFirst few rows of augmented data:")
display(augmented_non_flooding_df.head())

# Save to CSV file
output_path = "non_flooding_events_augmented.csv"
augmented_non_flooding_df.to_csv(output_path, index=False)
print(f"\nAugmented data saved to: {output_path}")


Augmenting 9500 flooding events with weather data...
This may take a while due to API rate limits...

Processed 50/9500 events...
Processed 100/9500 events...
Processed 150/9500 events...
Processed 200/9500 events...
Processed 250/9500 events...
Processed 300/9500 events...
Processed 350/9500 events...
Processed 400/9500 events...
Processed 450/9500 events...
Processed 500/9500 events...
Processed 550/9500 events...
Processed 600/9500 events...
Processed 650/9500 events...
Processed 700/9500 events...
Processed 750/9500 events...
Processed 800/9500 events...
Processed 850/9500 events...
Processed 900/9500 events...
Processed 950/9500 events...
Processed 1000/9500 events...
Processed 1050/9500 events...
Processed 1100/9500 events...
Processed 1150/9500 events...
Processed 1200/9500 events...
Processed 1250/9500 events...
Processed 1300/9500 events...
Processed 1350/9500 events...
Processed 1400/9500 events...
Processed 1450/9500 events...
Processed 1500/9500 events...
Processed 1550/950

,YEAR,MONTH,BEGIN_DAY,BEGIN_TIME,BEGIN_LAT,BEGIN_LON,STATE,EVENT_TYPE,FLOOD_CAUSE,EVENT_NARRATIVE,...,wind_speed_10m_mean,cloud_cover_mean,relative_humidity_2m_mean,dew_point_2m_mean,precipitation_sum,rain_sum,snowfall_sum,pressure_msl_mean,soil_moisture_0_to_10cm_mean,elevation
0,2010,5,1,18,29.41000,-100.90000,TEXAS,Hail,NaN,NaN,...,10.253766,28.487219,60.979492,13.821892,0.000000,0.000000,0.0,1013.899902,0.326249,318.0
1,2015,4,11,2040,35.24000,-100.16000,TEXAS,Hail,NaN,Law enforcement received golf ball size hail 5...,...,16.718573,35.630985,75.888046,17.391108,0.000000,0.000000,0.0,1008.741272,0.273536,711.0
2,2001,3,14,1410,30.10000,-96.08333,TEXAS,Hail,NaN,NaN,...,8.252332,82.120888,85.798393,20.577196,1.197766,1.197766,0.0,1017.202942,0.259449,72.0
3,2001,10,12,1945,33.21667,-97.15000,TEXAS,Thunderstorm Wind,NaN,Large tree limbs were blown off trees.,...,23.663277,64.207565,78.247787,20.589581,0.267922,0.267922,0.0,1012.636353,0.207057,214.0
4,2016,4,27,45,30.41000,-98.74000,TEXAS,Thunderstorm Wind,NaN,A thunderstorm produced wind gusts estimated a...,...,17.052565,56.042210,29.054462,4.967474,0.000000,0.000000,0.0,1010.412048,0.146342,534.0



Augmented data saved to: non_flooding_events_augmented.csv


In [ ]:
### Get testing data for flooding map of texas counties

from datetime import datetime, timedelta

texas_county_centers = pd.read_csv("raw_data/Texas_Counties_Centroid_Map.csv")
BEGIN_TIME = 0
NUM_DATES_PER_COUNTY = 30

# Generate 30 evenly spaced dates across 2025
start_date = datetime(2025, 1, 1)
end_date = datetime(2025, 12, 31)
total_days = (end_date - start_date).days
# Create evenly spaced dates (including start and end)
date_intervals = [start_date + timedelta(days=int(total_days * i / (NUM_DATES_PER_COUNTY - 1))) 
                  for i in range(NUM_DATES_PER_COUNTY)]

# Create a DataFrame with the dates
dates_df = pd.DataFrame({
    'date': date_intervals
})
dates_df['YEAR'] = dates_df['date'].dt.year
dates_df['MONTH'] = dates_df['date'].dt.month
dates_df['BEGIN_DAY'] = dates_df['date'].dt.day
dates_df['BEGIN_TIME'] = BEGIN_TIME
dates_df = dates_df[['YEAR', 'MONTH', 'BEGIN_DAY', 'BEGIN_TIME']]

# Prepare county data
counties_df = texas_county_centers[["X (Lat)", "Y (Long)", "CNTY_NM"]].copy()
counties_df.rename(columns={"X (Lat)": "BEGIN_LAT", "Y (Long)": "BEGIN_LON", "CNTY_NM": "NAME"}, inplace=True)

# Create a key for cross join
dates_df['key'] = 1
counties_df['key'] = 1

# Cross join to create all combinations (each county with all 30 dates)
testing_data = counties_df.merge(dates_df, on='key', how='outer').drop('key', axis=1)

# Reorder columns to match expected format
testing_data = testing_data[["YEAR", "MONTH", "BEGIN_DAY", "BEGIN_TIME", "BEGIN_LAT", "BEGIN_LON", "NAME"]]

print(f"Created {len(testing_data)} rows ({len(texas_county_centers)} counties × {NUM_DATES_PER_COUNTY} dates)")
testing_data.head()


,YEAR,MONTH,BEGIN_DAY,BEGIN_TIME,BEGIN_LAT,BEGIN_LON,NAME
0,NaN,NaN,NaN,NaN,33.974085,-99.778711,Foard
1,NaN,NaN,NaN,NaN,30.266361,-98.399741,Blanco
2,NaN,NaN,NaN,NaN,33.607504,-102.343092,Hockley
3,NaN,NaN,NaN,NaN,30.705666,-98.683874,Llano
4,NaN,NaN,NaN,NaN,34.530650,-101.734951,Swisher
